In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import time

# 크롬 드라이버 경로 설정
driver = webdriver.Chrome()

# 웹페이지로 이동
driver.get('https://portal.kfb.or.kr/consumer/branch.php')

# 페이지 로드 대기
time.sleep(3)

searchloc = '서울특별시'

search_area = driver.find_element(By.XPATH, r'//*[@id="s_que3"]')
search_area.send_keys(searchloc)
driver.find_element(By.XPATH, r'//*[@id="Content"]/div[3]/form/div/ul/li[2]/span[5]/a').send_keys(Keys.ENTER)  # 돋보기 클릭

time.sleep(5)

# 3. 검색 결과가 있는 표 찾기
# 데이터를 저장할 리스트 생성
all_data = []

# 2. 페이지 수집 루프
page = 1
while True:
    # 페이지가 로드될 시간을 확보
    time.sleep(3)
    
    # 3. 현재 페이지의 표 데이터를 가져오기
    try:
        table = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[3]/div[3]/table')
        html_content = table.get_attribute('outerHTML')
        soup = BeautifulSoup(html_content, 'html.parser')
        rows = soup.find_all('tr')
        
        # 행에서 데이터를 추출하여 리스트에 저장
        for row in rows:
            cells = row.find_all('td')
            if len(cells) > 0:
                bank = cells[0].text.strip()
                branch = cells[1].text.strip()
                address = cells[2].text.strip()
                phone = cells[3].text.strip()
                all_data.append([bank, branch, address, phone])
        
        print(f"Page {page} data collected")
    except:
        print(f"Error occurred on page {page}")
        break

    # 4. 다음 페이지로 이동
    try:
        # 페이지 번호 10 이후에는 '다음' 버튼 클릭
        if page % 10 == 0:
            next_button = driver.find_element(By.LINK_TEXT, '다음')
        else:
            next_button = driver.find_element(By.LINK_TEXT, str(page + 1))
        
        next_button.click()
        page += 1
    
    except:
        # 더 이상 '다음' 버튼이 없으면 종료
        print("No more pages.")
        break

# 5. pandas 데이터프레임으로 변환
df = pd.DataFrame(all_data, columns=['은행', '지점', '주소', '전화번호'])

# 데이터프레임 출력
print(df)

# 크롬 드라이버 종료
driver.quit()



Page 1 data collected
Page 2 data collected
Page 3 data collected
Page 4 data collected
Page 5 data collected
Page 6 data collected
Page 7 data collected
Page 8 data collected
Page 9 data collected
Page 10 data collected
Page 11 data collected
Page 12 data collected
Page 13 data collected
Page 14 data collected
Page 15 data collected
Page 16 data collected
Page 17 data collected
Page 18 data collected
Page 19 data collected
Page 20 data collected
Page 21 data collected
Page 22 data collected
Page 23 data collected
Page 24 data collected
Page 25 data collected
Page 26 data collected
Page 27 data collected
Page 28 data collected
Page 29 data collected
Page 30 data collected
Page 31 data collected
Page 32 data collected
Page 33 data collected
Page 34 data collected
Page 35 data collected
Page 36 data collected
Page 37 data collected
Page 38 data collected
Page 39 data collected
Page 40 data collected
Page 41 data collected
Page 42 data collected
Page 43 data collected
Page 44 data collect

In [32]:
df.drop('전화번호', axis=1, inplace=True)

In [34]:
df.to_csv('bank_crol.csv')

## 크롤링한 은행 지점 데이터 전처리

In [1]:
import pandas as pd

In [18]:
df = pd.read_csv('bank_crol.csv', index_col='Unnamed: 0')

In [3]:
df

,은행,지점,주소
0,BNK경남은행,가산디지털,서울특별시 금천구 가산동 가산디지털 1로
1,BNK경남은행,강남,서울특별시 강남구 대치동 영동대로 421
2,BNK경남은행,마곡,서울특별시 강서구 마곡동 공항대로 227
3,BNK경남은행,서울영업부,서울특별시 중구 서소문동 세종대로9길 42
4,BNK경남은행,여의도,서울특별시 영등포구 여의도동 국제금융로2길 36
...,...,...,...
1376,한국씨티은행,청담센터,서울특별시 강남구 청담동 압구정로
1377,한국씨티은행,커머셜글로벌기업금융센터,서울특별시 종로구 신문로2가 새문안로
1378,한국씨티은행,커머셜기업금융1센터,서울특별시 종로구 신문로2가 새문안로
1379,한국씨티은행,커머셜기업금융2센터,서울특별시 강남구 대치동 테헤란로


In [4]:
import pandas as pd
import numpy as np

df['은행'].unique()

array(['BNK경남은행', 'BNK부산은행', 'IBK기업은행', 'KB국민은행', 'KDB산업은행', 'NH농협은행',
       'SC제일은행', 'Sh수협은행', 'iM뱅크(구 대구은행)', '광주은행', '우리은행', '전북은행', '제주은행',
       '하나은행', '한국씨티은행'], dtype=object)

In [5]:
df.isna().sum()

은행    0
지점    0
주소    0
dtype: int64

In [6]:

# 중복된 단어 제거 함수
def remove_duplicates(address):
    words = address.split()  # 공백을 기준으로 단어를 나눔
    seen = set()  # 이미 본 단어를 저장할 집합
    result = []  # 결과를 저장할 리스트
    
    for word in words:
        if word not in seen:  # 단어가 처음 나타나면 리스트에 추가
            result.append(word)
            seen.add(word)
    
    return ' '.join(result)  # 다시 공백을 기준으로 단어를 합침



In [7]:
# '주소' 열에 대해 중복 단어 제거 적용
df['주소'] = df['주소'].apply(remove_duplicates)

In [19]:
def remove_dong_words(address):
    words = address.split()  # 공백을 기준으로 단어를 나눔
    result = []  # 결과를 저장할 리스트
    
    for word in words:
        if '동' not in word:  # '동'이 포함되지 않은 단어만 추가
            result.append(word)
    
    return ' '.join(result)  # 다시 공백을 기준으로 단어를 합침

# 동 이름 제거
df['주소'] = df['주소'].apply(remove_dong_words)

In [20]:
df

,은행,지점,주소
0,BNK경남은행,가산디지털,서울특별시 금천구 가산디지털 1로
1,BNK경남은행,강남,서울특별시 강남구 421
2,BNK경남은행,마곡,서울특별시 강서구 공항대로 227
3,BNK경남은행,서울영업부,서울특별시 중구 세종대로9길 42
4,BNK경남은행,여의도,서울특별시 영등포구 국제금융로2길 36
...,...,...,...
1376,한국씨티은행,청담센터,서울특별시 강남구 압구정로
1377,한국씨티은행,커머셜글로벌기업금융센터,서울특별시 종로구 신문로2가 새문안로
1378,한국씨티은행,커머셜기업금융1센터,서울특별시 종로구 신문로2가 새문안로
1379,한국씨티은행,커머셜기업금융2센터,서울특별시 강남구 테헤란로


In [21]:
num_remain = df.index[df['은행']=='KB국민은행'].to_list()

In [22]:
# 국민은행 행 지우기
df.drop(num_remain, axis=0, inplace=True)

In [23]:
df[df['은행']=='KB국민은행']

,은행,지점,주소


## 은행 좌표 추출

In [12]:
import requests
import pandas as pd

# 네이버 API 클라이언트 정보
client_id = "3fmd7xia0h"
client_secret = "uy5gOT87pjBgIeAmXCH2dpXVBKK090HceaprqeKL"

# API 요청 함수 정의
def get_coordinates(address):
    endpoint = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"
    url = f"{endpoint}?query={address}"

    # API 요청 헤더
    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
    }

    # API 호출 및 좌표 추출
    res = requests.get(url, headers=headers)
    if res.status_code == 200:
        data = res.json()
        if data['status'] == 'OK' and data['meta']['totalCount'] > 0:
            x = data['addresses'][0]['x']
            y = data['addresses'][0]['y']
            return pd.Series([x, y])
    return pd.Series([None, None])

In [25]:
# 주소 열에 대해 좌표를 적용하여 x, y 좌표 열 추가
df[['x', 'y']] = df['주소'].apply(get_coordinates)

# 결과 확인
df.head()

,은행,지점,주소,x,y
0,BNK경남은행,가산디지털,서울특별시 금천구 가산디지털 1로,None,None
1,BNK경남은행,강남,서울특별시 강남구 421,None,None
2,BNK경남은행,마곡,서울특별시 강서구 공항대로 227,126.8326818,37.5595617
3,BNK경남은행,서울영업부,서울특별시 중구 세종대로9길 42,126.9738508,37.5616585
4,BNK경남은행,여의도,서울특별시 영등포구 국제금융로2길 36,126.9237921,37.5236576


In [29]:
df[df['x'].isna()]['은행'].unique()

array(['BNK경남은행', 'BNK부산은행', 'IBK기업은행', 'KDB산업은행', 'NH농협은행', 'SC제일은행',
       'Sh수협은행', '우리은행', '전북은행', '하나은행', '한국씨티은행'], dtype=object)

In [35]:
df = df[(df['은행']=='NH농협은행')|(df['은행']=='IBK기업은행')|(df['은행']=='KDB산업은행')|(df['은행']=='SC제일은행')|(df['은행']=='Sh수협은행')|(df['은행']=='우리은행')|(df['은행']=='하나은행')]

In [36]:
df.reset_index(inplace=True)
df.drop('index',axis=1, inplace=True)

C:\Users\swc08\AppData\Local\Temp\ipykernel_23412\3085771687.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop('index',axis=1, inplace=True)


In [14]:
df[df['x'].isna()].head(70)

,은행,지점,주소,x,y
0,IBK기업은행,LG광화문,서울특별시 종로구 새문안로 58,NaN,NaN
12,IBK기업은행,강남구청,서울특별시 강남구 429,NaN,NaN
24,IBK기업은행,공릉역,서울특별시 노원구 1073,NaN,NaN
36,IBK기업은행,남구로,서울특별시 구로구 126-2,NaN,NaN
37,IBK기업은행,남대문,서울특별시 중구 세종대로 17,NaN,NaN
...,...,...,...,...,...
592,우리은행,목동지점,서울특별시 양천구 159-1,NaN,NaN
593,우리은행,무역센터금융센터,서울특별시 강남구 511,NaN,NaN
613,우리은행,삼성동금융센터,서울특별시 강남구 432,NaN,NaN
620,우리은행,상계동지점,서울특별시 노원구 1373,NaN,NaN


In [39]:
df.to_csv('bank_crol.csv')

In [35]:
import pandas as pd
df = pd.read_csv('bank_crol.csv', index_col='Unnamed: 0',encoding='cp949')

In [36]:
df['은행'].unique()

array(['IBK기업은행', 'KDB산업은행', 'NH농협은행', 'SC제일은행', 'Sh수협은행', '우리은행', '하나은행'],
      dtype=object)

In [37]:
bank_1 = pd.read_csv('bank_crol.csv', index_col='Unnamed: 0', encoding='cp949')
bank_2 = pd.read_csv('bank_crol_2.csv', index_col='Unnamed: 0')

In [38]:
bank_1 = bank_1.loc[:643]

In [39]:
bank_2 = bank_2.loc[644:]

In [40]:
bank = pd.concat([bank_1, bank_2],axis=0)

In [41]:
num = bank.index[bank['x'].isna()].to_list()

In [42]:
bank.drop(num, axis=0, inplace=True)

In [43]:
bank[bank['x'].isna()]

,은행,지점,주소,x,y


In [44]:
kookmin = pd.read_csv('kookmin_bank.csv', index_col='Unnamed: 0')

In [45]:
bank = pd.concat([bank, kookmin],axis=0)

In [49]:
bank.reset_index(inplace=True)

In [50]:
bank.drop('index',axis=1,inplace=True)

In [51]:
bank.to_csv('bank_crol_1007.csv')

In [52]:
bank

,은행,지점,주소,x,y
0,IBK기업은행,LG광화문,서울특별시 종로구 신문로2가 새문안로 58,126.971586,37.569311
1,IBK기업은행,MBC,서울특별시 마포구 성암로 267,126.890989,37.581123
2,IBK기업은행,가락동,서울특별시 송파구 중대로 97,127.119352,37.493175
3,IBK기업은행,가산IT밸리,서울특별시 금천구 가산디지털2로 53,126.881148,37.472973
4,IBK기업은행,가산동,서울특별시 금천구 가산디지털2로 184,126.878042,37.484256
...,...,...,...,...,...
1310,국민은행,목동PB센터,"서울특별시 양천구 목동동로 381,2층(목동)",126.879045,37.535736
1311,국민은행,잠실롯데PB센터,"서울특별시 송파구 올림픽로 240,4층(잠실동,롯데월드)",127.098164,37.511079
1312,국민은행,스타시티PB센터,"서울특별시 광진구 능동로 90,A동2층(자양동)",127.070972,37.538630
1313,국민은행,강남스타PB센터,"서울특별시 강남구 테헤란로 152,21층(역삼동,강남파이낸스센터)",127.036546,37.500026
